// 参考
https://www.kaggle.com/code/kleedg/cmi-piu-lee-dong-gi


In [ ]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns
import plotly.subplots as sp
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor
from colorama import Fore, Style
from IPython.display import clear_output
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None